# TFY4325/FY8904: Computational Physics (spring 2024)
## Assignment 2: The World of Quantum Mechanics

In this assignment we will study three different quantum mechanical systems, described by onedimensional Schrödinger equations. 

The first problem is the simple and well-known particle in a box, where we focus on scaling to dimensionless equations and comparing computational results to the exact
ones. 

The second problem describes a particle in a box with a potential barrier, where we do not have
many analytical results. 

In the third problem we learn how we can study a simple time-dependent quantum mechanical
platform. By considering a time-dependent Schr¨odinger equation for oscillating adjacent quantum wells
we find a time-dependent wavefunction that result in an integral equation. Finding the time-dependent
wavefunction, we predict the evolution of non-interacting particles that might exist in this system.

# 1. Particle in a box and dimensionless variables

The Schrödinger equation reads as follows

$$
i\hbar \frac{\partial}{\partial t}\Psi = \hat{H}\Psi
$$
which is the baseline of quantum mechanics, describing how a system evolves in time given the Hamilationian operator $\hat{H}$ for the system.

Given some initial condition $\Psi_0(x)=\Psi(x,t = 0)$, and a time independent Hamiltonian, we get a general solution

$$
\Psi = exp(-\frac{it}{\hbar}\hat{H})\Psi_0
$$

with the expontentional of the Hamiltionian being the series expansion. This equation describes the time evoolution of the system, and requires the eigenfunctions $\psi_n$ of the Hamiltionian for it to be useful. They can be extracted through the time-independent Schrödinger equation

$$
\hat{H}\psi_n = E_n \psi_n
$$

$E_n$ being the energy of the state $\psi_n$.

We want to study a single particle with mass $m$ confined in some 2D box with walls at $x=0$, and $x=L$.

The boundary conditions required for the wave function when the particle is confined in the box can be seen by the potential that describes the box

$$
V(x) = 
$$
